In [1]:
#importing the modules/libraries
import cv2        #for taking the video feed
import numpy as np  #for mathematical operations
from time import sleep   #it will insure that our loop can take only the defined frames per second 

In [2]:
min_width=80  #minimum width of rectangle across the Vehicle 
min_hight=80  #minimum hight of rectangle across the vehicle

pixel_error=6  #Permitted pixel error for defining the center

line_pos=550  #line position(depth) in the frame according to captured video
fps= 60  #frames per second

detect_center = []  #list that will contain the cordinates of center of rectangle
cars= 0  #initializing the variable for counting

In [3]:
def catch_center(x, y, w, h):
    x1 = w //2  
    y1 = h//2
    cx = x + x1
    cy = y + y1 
    return cx,cy

In [5]:
cap = cv2.VideoCapture(r'C:\Users\dell\Documents\traffic_counter\video.mp4') #capturing the video
b_sub= cv2.bgsegm.createBackgroundSubtractorMOG()  #creating the background subtractor

In [6]:
while True:     #loop for taking continous frames
    ret , frame1 = cap.read()  #reading the frames 
    time = float(1/fps)      
    sleep(time)                #allowing only 60 frames per second 
    grey = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)     #converting BGR(blue,green,red) into black in white or gray frame
    blur = cv2.GaussianBlur(grey,(3,3),5)              #applying blurr filter
    img_sub = b_sub.apply(blur)                        #creating background subtracted image
    #applying dilation filter
    dilation =cv2.dilate(img_sub,np.ones((5,5)))       
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))    
    dilated = cv2.morphologyEx (dilation, cv2. MORPH_CLOSE , kernel)
    dilated = cv2.morphologyEx (dilated, cv2. MORPH_OPEN , kernel)
    
    contour,h = cv2.findContours(dilated,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) #creating counters to detect the vehicles
    cv2.line(frame1, (25, line_pos), (1200, line_pos), (255,127,0),3)  #drawing line at the depth of 550 on the frame
    for(i,c) in enumerate(contour):                                    #loop for finding the cordinates of rectangle from contour
        (x,y,w,h) = cv2.boundingRect(c)
        if not ((w >= min_width) and (h >= min_hight)): #if contour is not valid the contiue the loop otherwise proceed
            continue

        cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,255,0),2)        #drawing a valid rectangle on the frame
        center = catch_center(x, y, w, h)                        #calculating the center of rectangle from above function
        detect_center.append(center)                             #adding crdinates in the detect_center list
        cv2.circle(frame1, center, 4, (0, 0,255), -1)            #marking center of rectangle    

        for (x,y) in detect_center:                                     #loop for counting the cars
            if y<(line_pos+pixel_error) and y>(line_pos-pixel_error):     #if center of rectangle lie between permissible upper and lower bound of line
                cars+=1                                                  #then increase the no of cars detected
                cv2.line(frame1, (25, line_pos), (1200, line_pos), (0,127,255), 3)  #once car is detected the again draw the same line for other cars
                detect_center.remove((x,y))                                                 #once the center is counted the remove it from the list
                print("Cars detected so far: "+str(cars))                          #output
       
    cv2.putText(frame1, "Vehicals: "+str(cars), (450, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255),5)       #to show output on video frames 
    cv2.imshow("Video Original" , frame1)                                                                   #the original video frame
    cv2.imshow("Detectar",dilated)                                                          #processed video frame

    if cv2.waitKey(1) == ord('e'):                                                        #press "e" to exit
        break
    
cv2.destroyAllWindows()
cap.release()                                                                   #to release the video

Cars detected so far: 1
Cars detected so far: 2
Cars detected so far: 3
Cars detected so far: 4
Cars detected so far: 5
Cars detected so far: 6
Cars detected so far: 7
